# Quantum Chemistry Tutorial 3 - Excited states

1. **S. G. Nana Engo**, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
2. **J-P. Tchapet Njafa**, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
       
January 2024

## Introduction

L'une des applications marquantes de la chimie quantique, tant dans le milieu universitaire que dans l'industrie, est l'étude de l'interaction de la lumière avec la matière. L'absorption (resp. émission) d'un photon par une molécule peut promouvoir (resp. rétrograder) un électron d'un état électronique inférieur (resp. supérieur) à un état électronique d'énergie supérieure (resp. inférieure). La longueur d'onde des photons (c'est-à-dire l'énergie) requise pour que ces transitions se produisent est déterminée par la différence entre les deux états électroniques respectifs. Par conséquent, il est impératif de pouvoir calculer avec précision les énergies des états fondamental et excité afin d'étudier les interactions lumière/matière. Ces différences énergétiques jouent un rôle central dans de nombreuses technologies telles que les panneaux solaires, les diodes électroluminescentes (DEL ou en anglais, light-emitting diodes (LED)), les écrans et les colorants.

![Abs_emiss.png](Graphics/Abs_emiss.png)

Pour être plus concret, un colorant doit émettre de la lumière dans une région étroite du spectre visible pour être approprié à son objectif, c'est-à-dire qu'il doit présenter une longueur d'onde spécifique. Un autre exemple est celui des panneaux solaires, où le spectre d'absorption d'une molécule est ajusté via une fonctionnalisation chimique pour s'adapter au spectre d'émission solaire afin d'optimiser l'efficacité de la production d'énergie. Nous montrons ici un exemple de spectre pour la molécule BODIPY ($C_9H_7BF_2N_2$, Boron Dipyrromethene), une molécule largement utilisée pour les colorants fluorescents. La méolécule BODIPY absorbe la lumière à une longueur d'onde inférieure (énergie plus élevée) et émet de la lumière à une longueur d'onde plus élevée (énergie plus faible). Pour calculer ce spectre, il faut calculer les énergies des états fondamental et excité et calculer leurs intensités. Le spectre d'absorption du BODIPY le plus simple est présenté ci-dessous. Différentes longueurs d'onde d'absorption et d'émission peuvent être ciblées en remplaçant les atomes d'hydrogène par différents groupes fonctionnels [J. Chimique. Phys. 155, 244102 (2021)](https://aip.scitation.org/doi/10.1063/5.0076787).

![BODIPY](Graphics/bodipy_absorption.png)

Comme il existe un très grand nombre de composés à considérer, la prévision des spectres d'absorption/émission UV-visible serait un atout précieux pour la communauté scientifique.

Pour parvenir à une compréhension complète de l'interaction de la lumière avec une molécule, la communauté de la chimie quantique a travaillé sur plusieurs algorithmes. En général, il faut calculer les structures électroniques moléculaires pertinentes pour prédire l'absorption/l'émission de la lumière UV. Ce notebook montre comment PySFC permet les calculs d'états excités en implémentant l'algorithme la TDDFT-TDA. Le cas d'utilisation ici est Li $_2$ par souci de simplicité. Ce calcul être étendu à des systèmes beaucoup plus grands tels que la molécule BODIPY ci-dessus.

Il convient de noter que même avec tous les états excités calculés, des effets non triviaux peuvent se produire (effet de solvatation, changement de géométrie, etc.) dans lesquels tous modifient la forme d'un spectre. Dans ce notebook, nous n'expliquons pas comment ces effets sont pris en compte, mais les calculs présentés ici constituent les premières étapes nécessaires vers le calcul des états excités.


## Théorie de la fonctionnelle de la densité dépendante du temps - TDDFT

En utilisant la théorie des perturbations dépendant du temps du premier ordre au sein de la théorie KS-DFT, on obtient les équations non Hermitiennes TDDFT pour les énergies d'excitation,
$$
     \begin{pmatrix}
         \mathbf{A} & \mathbf{B} \\
         \mathbf{B}^\ast & \mathbf{A}^\ast
     \end{pmatrix}
     \begin{pmatrix}  \mathbf{X} \\ \mathbf{Y}
     \end{pmatrix} = \omega
     \begin{pmatrix}
         \mathbf{1} & \mathbf{0} \\
         \mathbf{0} & -\mathbf{1}
     \end{pmatrix}
     \begin{pmatrix}  \mathbf{X} \\ \mathbf{Y}
     \end{pmatrix},
$$    
où 
* $\mathbf{A}$ et $\mathbf{B}$ sont les orbitales les Hessiennes qui apparaissent également dans l'analyse de stabilité pour les états de référence ,
* $\omega$ est l'énergie d'excitation, et
* $\mathbf{X}$ et $\mathbf{Y}$ représentent la réponse de la matrice de densité ou l'opérateur statistique.

Dans les cas où le système possède un état fondamental dégénéré ou présente des instabilités triplet, les algorithmes utilisés pour résoudre le
les équations ci-dessus peuvent être instables. Ceci peut être résolu en appliquant l'approximation de Tamm-Dancoff (TDA), où l'on néglige simplement les matrices $\mathbf{B}$ et $\mathbf{Y}$. Il en résulte un problème Hermitien aux valeurs propres
$$
\mathbf{AX} = \omega\mathbf{X}.
$$

Il est à noter que lorsque TDA est appliquée, dans la TDDFT, la réponse linéaire du potentiel de corrélation d'échange (XC) conduit à une dérivée d'ordre 2 de la fonctionnelle XC, qui n'apparaît pas dans la DFT de l'état fondamental.


## Propriétés des transitions

Les énergies d’excitation ne rendent compte ne sont pas suffisantes comprendre la physique du problème. Nous avons besoin de moments de transition pour comprendre si une excitation est autorisée (lumineuse) ou non (sombre).

Les énergies d'excitation et les vecteurs propres peuvent être utilisés pour calculer les moments de transition, tels que les moments dipolaires de transition électrique et magnétique, et les intensités spectroscopiques, telles que les forces d'oscillateur et les forces de rotation.
Par exemple, le moment dipolaire électrique de transition dans la jauge de longueur est :

$$
  f = \frac23 \omega_n \sum_{u=x,y,z}\sum_{ia}|(\mathbf{X}_n+\mathbf{Y}_n)_{ia}\mu_{ai, u}|^2.
$$


## Calculs avec PySCF

Le système moléculaire que nous utilisons comme exemple dans ce notebook est le Li $_2$ proche de sa géométrie d'équilibre [voir les données expérimentales sur la géométrie en phase gazeuse](https://cccbdb.nist.gov/expgeom2x.asp). 
<!-- Le calcul complet des énergies Li $_2$ serait non trivial et très coûteux en calcul ; nous nous limitons donc à un espace actif de 2 électrons sur 2 orbitales qui impliquent 4 qubits lorsqu'ils sont mappés sur un hamiltonien de qubits en utilisant la cartographie de Jordan-Wigner. Cependant, ce petit problème a encore des effets non triviaux, rendus particulièrement évidents dans la section [2.7] (#27). Nous définissons deux objets molécules :

- `mol_li2` défini comme la configuration de l'état fondamental avec 2 électrons dans le HOMO.
- `mol_li2_t` défini comme la configuration triplet avec un électron alpha dans chacun des HOMO et LUMO. -->

### Calculs DFT

In [3]:
from pyscf import gto, scf, dft, tddft

mol_xyz = 'Li 0 0 0; Li 0 0 3.0' # Ref: https://cccbdb.nist.gov/expgeom2x.asp
mol = gto.Mole()
mol.build(
    atom = mol_xyz,  
    basis = '6-31g(d,p)',
    symmetry = True,
)

mf = dft.RKS(mol)
mf.xc = 'b3lyp'
mf.kernel()
mf.analyze()

/home/taamangtchu/mambaforge/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


converged SCF energy = -15.0123443659292
Wave-function symmetry id = 0
occupancy for each irrep:    A1g E1gx E1gy  A1u E1uy E1ux E2gx E2gy E2uy E2ux
                               2    0    0    1    0    0    0    0    0    0
**** MO energy ****
MO #1 (A1g #1), energy= -2.00064068373249 occ= 2
MO #2 (A1u #1), energy= -2.00056524182952 occ= 2
MO #3 (A1g #2), energy= -0.129344073811441 occ= 2
MO #4 (A1u #2), energy= -0.0549025752066173 occ= 0
MO #5 (E1uy #1), energy= -0.0229060330406869 occ= 0
MO #6 (E1ux #1), energy= -0.0229060330406869 occ= 0
MO #7 (A1g #3), energy= -0.0130314467468124 occ= 0
MO #8 (E1gx #1), energy= 0.0221211001313741 occ= 0
MO #9 (E1gy #1), energy= 0.0221211001313741 occ= 0
MO #10 (A1u #3), energy= 0.067043270972293 occ= 0
MO #11 (A1g #4), energy= 0.104835858582985 occ= 0
MO #12 (E1uy #2), energy= 0.115765883514776 occ= 0
MO #13 (E1ux #2), energy= 0.115765883514776 occ= 0
MO #14 (A1g #5), energy= 0.140875174614696 occ= 0
MO #15 (E1gx #2), energy= 0.146347857475865 o

((array([1.99954536e+00, 9.83322257e-01, 1.69538208e-03, 4.61669976e-35,
         4.45462075e-34, 9.77563437e-03, 1.69942833e-34, 3.60472887e-36,
         5.24425576e-03, 1.04862288e-36, 5.93670295e-36, 4.17108251e-04,
         2.32408346e-37, 3.30946642e-40, 1.99954536e+00, 9.83322257e-01,
         1.69538208e-03, 2.18681967e-35, 4.13793879e-36, 9.77563437e-03,
         2.79222572e-36, 7.83006727e-35, 5.24425576e-03, 6.51337956e-37,
         2.73637886e-36, 4.17108251e-04, 3.73633987e-35, 6.18098685e-40]),
  array([4.44089210e-16, 1.77635684e-15])),
 array([0., 0., 0.]))

### Calculs des états excités singulets avec la TDDFT

Effectuons les calculs sans et avec l'approximation TDA.

In [2]:
mytd = tddft.TDDFT(mf)
mytd.kernel()
mytd.analyze(verbose=4)

Excited State energies (eV)
[1.97453503 2.72071313 2.72071321]

** Singlet excitation energies and oscillator strengths **
Excited State   1:  A1u      1.97454 eV    627.92 nm  f=0.5099
       3 -> 4        -0.71952
Excited State   2:  ???      2.72071 eV    455.70 nm  f=0.5222
       3 -> 5         0.21815
       3 -> 6         0.67924
Excited State   3:  ???      2.72071 eV    455.70 nm  f=0.5222
       3 -> 5        -0.67924
       3 -> 6         0.21815

** Transition electric dipole moments (AU) **
state          X           Y           Z        Dip. S.      Osc.
  1         3.2465     -0.0000      0.0000     10.5396      0.5099
  2         0.0000      2.6649      0.8559      7.8339      0.5222
  3         0.0000      0.8559     -2.6649      7.8339      0.5222

** Transition velocity dipole moments (imaginary part, AU) **
state          X           Y           Z        Dip. S.      Osc.
  1         0.2371     -0.0000     -0.0000      0.0562      0.5166
  2         0.0000      0.28

In [3]:
mytda = tddft.TDA(mf)
mytda.kernel()
mytda.analyze(verbose=4)

Excited State energies (eV)
[2.21611113 2.85924466 2.85924466]

** Singlet excitation energies and oscillator strengths **
Excited State   1:  A1u      2.21611 eV    559.47 nm  f=0.8699
       3 -> 4         0.69358
       3 -> 10       -0.13053
Excited State   2:  ???      2.85924 eV    433.63 nm  f=0.7240
       3 -> 5        -0.24497
       3 -> 6        -0.66021
Excited State   3:  ???      2.85924 eV    433.63 nm  f=0.7240
       3 -> 5        -0.66021
       3 -> 6         0.24497

** Transition electric dipole moments (AU) **
state          X           Y           Z        Dip. S.      Osc.
  1        -4.0027     -0.0000     -0.0000     16.0214      0.8699
  2         0.0000     -3.0141     -1.1184     10.3354      0.7240
  3        -0.0000      1.1184     -3.0141     10.3354      0.7240

** Transition velocity dipole moments (imaginary part, AU) **
state          X           Y           Z        Dip. S.      Osc.
  1        -0.1766     -0.0000     -0.0000      0.0312      0.255

In [4]:
# Singlet excitation energies
ee_singlets = mytda.e

# Singlet oscillators strength
OS_singlets = mytda.oscillator_strength(gauge='length')

### Calculs des états excités triplets avec la TDDFT

Par défaut, seuls les états excités singulets sont calculés. Afin de calculer les états excités des triplets, il faut définir l'attribut *singlet* sur `False`.

In [5]:
mytda.singlet = False
mytda.kernel()
mytda.analyze(verbose=4)

Excited State energies (eV)
[0.87598355 1.53621236 1.53621236]

** Triplet excitation energies and oscillator strengths **
Excited State   1:  A1u      0.87598 eV   1415.37 nm  f=0.3652
       3 -> 4         0.70464
Excited State   2:  ???      1.53621 eV    807.08 nm  f=0.3805
       3 -> 5        -0.11914
       3 -> 6        -0.69329
Excited State   3:  ???      1.53621 eV    807.08 nm  f=0.3805
       3 -> 5         0.69329
       3 -> 6        -0.11914

** Transition electric dipole moments (AU) **
state          X           Y           Z        Dip. S.      Osc.
  1        -4.1253      0.0000     -0.0000     17.0181      0.3652
  2         0.0000     -3.1335     -0.5385     10.1091      0.3805
  3        -0.0000     -0.5385      3.1335     10.1091      0.3805

** Transition velocity dipole moments (imaginary part, AU) **
state          X           Y           Z        Dip. S.      Osc.
  1        -0.1965      0.0000     -0.0000      0.0386      0.7994
  2         0.0000     -0.24

In [6]:
# Triplet excitation energies
ee_triplets = mytda.e

### Calculs des propriétés

In [9]:
import os
import numpy as np
from pyscf.data import nist

au2ev = nist.HARTREE2EV

# Index of HOMO and LUMO
lumo_idx = mf_pyscf.mo_occ.tolist().index(0.)
homo_idx = lumo_idx - 1

# Calculate the gap Homo-LUMO
E_HOMO = mf_pyscf.mo_energy[homo_idx]*au2ev
E_LUMO = mf_pyscf.mo_energy[lumo_idx]*au2ev
E_g = abs(E_HOMO - E_LUMO)

# fluorescence energy
f_energy = min(ee_singlets)

# Singlet-Tiplet gap
gap_ST = min(ee_singlets) - min(ee_triplets)

# Oscillator strength
OStr = OS_singlets[0]

# Lifetime calculation in au
c = nist.LIGHT_SPEED # in au
time_au = 2.4188843265857E-17 # in au

tau_au = c**3 / (2 * (f_energy)**2 * OStr)
tau = tau_au * time_au

# Multi-Objective function
mobj = OStr - gap_ST * au2ev - np.abs(f_energy * au2ev - 3.2)


NameError: name 'ee_singlets' is not defined

* DataFrame and saving in file


In [9]:
import pandas as pd

# Create the results dataframe
list_results_TDA = [E_HOMO, E_LUMO, E_g, f_energy* au2ev, gap_ST* au2ev, OStr, tau* 1E+9, mobj]
dict_results_TDA = {'eV/ns': list_results_TDA}

df_TDA = pd.DataFrame(dict_results_TDA,
                    index = ['Homo energy', 'LUMO energy', 'Gap energy', 
                             'Fluorescence energy',
                             'Singlet-Triplet gap',
                             'Oscillator strength',
                             'Lifetime',
                             'Multi-Obj'])

df_TDA

,a.u.,eV/ns
Fluorescence energy,8.144058e-02,2.216111
Singlet-Triplet gap,4.924878e-02,1.340128
Oscillator strength,8.698589e-01,0.869859
Lifetime,5.394592e-09,5.394592
Multi-Obj,-1.454158e+00,-1.454158


## Spectre UV/Vis

Dans cette section, nous allons calculer les énergies d'excitation de la molécule 1,2-oxazole ($C_3H_3NO$) et tracer son spectre UV/Vis. Il est à noter que la molécule 1,2-oxazole peut être utile comme pesticide, fongicide agrochimique et insecticide.

In [5]:
import py3Dmol

# PySCF molecule object
mol_xyz = """  
  C      1.0701      0.4341     -0.0336
  C      0.8115     -0.9049     -0.1725
  C     -0.6249     -1.0279     -0.0726
  H      1.9842      1.0231     -0.0364
  H      1.5156     -1.7176     -0.3255
  H     -1.2289     -1.9326     -0.1322
  O     -0.0873      1.1351      0.1422
  N     -1.1414      0.1776      0.1122"""

mol = gto.M(atom = mol_xyz,
  basis = '6-31g(d,p)', 
  verbose=3)

# 3D representation
xyz_view = py3Dmol.view(width=300,height=300)
xyz_view.addModel(mol.tostring(format="xyz"),'xyz')
xyz_view.setStyle({'stick':{}, "sphere":{"radius":0.4}})
xyz_view.setBackgroundColor('0xeeeeee')
xyz_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

* Définissons une distribution Gaussienne qui est un moyen approximatif de se rapprocher de la forme de la ligne pour chaque excitation. 

In [10]:
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))


* Créons une fonction pour calculer l'état fondamental avec la DFT, et les états excités avec la TD-DFT et effectuer l'analyse spectrale. 

In [11]:
def run_spectral_analysis(mol, xc="lda"):
    n_states=15
    spectral_width=0.1

    # Ground State DFT
    mf = dft.RKS(mol, xc=xc).run()

    # Excited State DFT
    mytd = tddft.TDDFT(mf)
    mytd.nstates = n_states
    mytd.max_space = 100
    mytd.max_cycle = 200
    mytd.kernel();
    mytd.analyze()
    osc_strengths = mytd.oscillator_strength()[:n_states-5]

    # Convolve lineshapes to make spectra
    energies_ev = mytd.e[:n_states-5]*au2ev
    x_range = np.linspace(energies_ev.min()*0.9, energies_ev.max()*1.1, num=1000)
    intensity = np.zeros(x_range.size)

    for e, f in zip(energies_ev, osc_strengths):
        intensity += gaussian(x_range, e, spectral_width) * f

    # Rough Normalization
    dx = (x_range[-1] - x_range[0])/x_range.size
    area = (intensity*dx).sum()
    intensity /= area


    return x_range, intensity

In [12]:
import time

data = {"Excitation Energy (eV)":[], "Intensity":[], "Exchange-Correlation Functional":[]}

xcs = ["lda", "pbe", "B3LYP"]

for xc in xcs:
    ti = time.time()
    x_range, intensity = run_spectral_analysis(mol, xc=xc)

    data["Excitation Energy (eV)"] += x_range.tolist()
    data["Intensity"] += intensity.tolist()
    data["Exchange-Correlation Functional"] += [xc]*x_range.size
    tf = time.time()
    print(f"Time for {xc.upper()} calculations: {tf-ti:.2f}\n")

df = pd.DataFrame(data)

converged SCF energy = -241.623913631131
Excited State energies (eV)
[5.66391225 6.25174474 6.90907571 7.20434454 7.60485404 8.05495939
 8.18570574 8.45233167 8.63425438 8.65019701 8.76724213 8.78967891
 8.85876696 9.36345593 9.52478924]

** Singlet excitation energies and oscillator strengths **
Excited State   1:      5.66391 eV    218.90 nm  f=0.0045
Excited State   2:      6.25174 eV    198.32 nm  f=0.0599
Excited State   3:      6.90908 eV    179.45 nm  f=0.0316
Excited State   4:      7.20434 eV    172.10 nm  f=0.0039
Excited State   5:      7.60485 eV    163.03 nm  f=0.0004
Excited State   6:      8.05496 eV    153.92 nm  f=0.0308
Excited State   7:      8.18571 eV    151.46 nm  f=0.0008
Excited State   8:      8.45233 eV    146.69 nm  f=0.0001
Excited State   9:      8.63425 eV    143.60 nm  f=0.1133
Excited State  10:      8.65020 eV    143.33 nm  f=0.0000
Excited State  11:      8.76724 eV    141.42 nm  f=0.0333
Excited State  12:      8.78968 eV    141.06 nm  f=0.1749
Excite

In [14]:
import plotly.express as px
fig = px.line(df, x="Excitation Energy (eV)", y="Intensity", markers=True, color="Exchange-Correlation Functional")
fig.show()